# Arguments in Smart Contracts
#### 06.5 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Interact with Smart Contracts using arguments

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Quick check of asset holdings, otherwise go to ...
- https://bank.testnet.algorand.network
- https://testnet.algoexplorer.io/dispenser

In [6]:
asset_holdings_df(algod_client,Alice['public'])

,amount,unit,asset-id,name,decimals
0,51.81699,ALGO,0,Algorand,6
1,200.00000,USDC,10458941,USDC,6
2,0.10000,WSC,66504861,WSC coin,2
3,0.00000,WSC,66505040,WSC coin,2
4,100.00000,ALICE,66712019,Alice's Tempcoin,1
5,100.00000,ALICE,66712340,Alice's Tempcoin,1
6,1.00000,ALICEART,69394953,Alice's First Portrait 001,0
7,0.00000,VtC,70161280,VoteCoin,2
8,0.00000,VtC,70166124,VoteCoin,2


### A few helper functions

In [ ]:
def payment_transaction(creator_mnemonic, amt, rcv, algod_client)->dict:
    params = algod_client.suggested_params()
    add = mnemonic.to_public_key(creator_mnemonic)
    key = mnemonic.to_private_key(creator_mnemonic)
    unsigned_txn = transaction.PaymentTxn(add, params, rcv, amt)
    signed = unsigned_txn.sign(key)
    txid = algod_client.send_transaction(signed)
    return txid

In [ ]:
def lsig_payment_txn(escrowProg, escrow_address, amt, rcv, algod_client):
    params = algod_client.suggested_params()
    unsigned_txn = transaction.PaymentTxn(escrow_address, params, rcv, amt)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

In [ ]:
def lsig_payment_txn_arg(escrowProg, escrow_address, amt, rcv, algod_client, arg0):
    params = algod_client.suggested_params()
    unsigned_txn = transaction.PaymentTxn(escrow_address, params, rcv, amt)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program, args=arg0)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

In [ ]:
def lsig_payment_txn_note(escrowProg, escrow_address, amt, rcv, algod_client,my_note):
    note = my_note.encode()
    params = algod_client.suggested_params()
    unsigned_txn = transaction.PaymentTxn(escrow_address, params, rcv, amt,None, note)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

In [ ]:
def lsig_payment_txn_ASA(escrowProg, escrow_address, amt, rcv, ASA_index, algod_client):
    params = algod_client.suggested_params()
    unsigned_txn = transaction.AssetTransferTxn(escrow_address, params, escrow_address, 0, ASA_index)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

## Numeric arguments
* Smart Signatures can work with multiple arguments
* Need to encode in Python and decode in PyTeal
* See "Passing parameters using the SDKs" https://developer.algorand.org/docs/get-details/dapps/smart-contracts/frontend/smartsigs/

## A mathematical quiz: find two numbers so that $x+y=10$
* Payout a small prize in Algo for the first to solve it
* Example solution 3+7 = 10

In [7]:
# Step 1: Conditions as a PyTeal
# (no security here)
quiz_cond = (
    Btoi(Arg(0))  + Btoi(Arg(1)) ==  Int(10)
)

# prepare random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = Txn.fee() <= Int(1000)

safety_cond = And(
    Txn.type_enum() == TxnType.Payment,
    # Txn.close_remainder_to() == Global.zero_address(),      # Quiz allows close out
    Txn.rekey_to() == Global.zero_address(),
    )

quiz_pyteal = And(
    quiz_cond,
    random_cond,
    fee_cond,
    safety_cond
)

In [9]:
# Step 2: Compile PyTeal -> Teal
quiz_teal = compileTeal(quiz_pyteal, Mode.Signature, version=3)
print(quiz_teal)

#pragma version 3
arg 0
btoi
arg 1
btoi
+
int 10
==
int 3275009137
int 3275009137
==
&&
txn Fee
int 1000
<=
&&
txn TypeEnum
int pay
==
txn RekeyTo
global ZeroAddress
==
&&
&&


In [10]:
# Step 3: Compile Teal -> Bytecode for AVM
Quiz = algod_client.compile(quiz_teal)

In [11]:
# Step 4: Alice is funding the quiz
# Step 4.1: prepare transaction
sp = algod_client.suggested_params()

# How much? Prize money of 1 Algo + TX fee
amt = int(1*1e6) + int(0.001*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, 
                             receiver=Quiz['hash'], amt=amt)

# Step 4.2 to 4.4: sign, send and wait ...
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19828526.
Waiting for round 19828526 to finish.
Waiting for round 19828527 to finish.
Transaction AOJ6WCETH6LSCURNNXPKKTDMEQJLRJZD4OLS264HRBNGHAGPCUBQ confirmed in round 19828528.


### Inform the public
* The quiz is now ready
* We need to communicate to the public ...

In [13]:
print('Address of the Quiz:      '+Quiz['hash'])
print('Program code for the Quiz: '+Quiz['result'])

Address of the Quiz:      TEBOHXDABX2SHT7UOKA5T3Y4WWMUXK7BUCGLVU52VH72WQOQMSS22CL2GA
Program code for the Quiz: AyAECvHY0pkM6AcBLRcuFwgiEiMjEhAxASQOEDEQJRIxIDIDEhAQ


#### Step 6: Bob submits a solution
* Using arguments for the smart contract (instead of notes)

In [14]:
# Step 6.1: prepare transaction
sp = algod_client.suggested_params()

# integer parameter
arg0 = (3).to_bytes(8, 'big')
arg1 = (7).to_bytes(8, 'big')
bob_solution_args = [arg0, arg1]                   # arguments need to be a list, 
                                                   # even if there is only 1 argument

txn = PaymentTxn(sender= Quiz['hash'], 
                 sp=sp, 
                 receiver=Bob['public'], 
                 amt=0,                            # <---- amount is zero
                 close_remainder_to=Bob['public']  # <---- This is how Bob gets the money
                )

# Steo 6.2: Sign
encodedProg = Quiz['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program, args=bob_solution_args)  # <------- HERE is where we add the arguments
stxn = LogicSigTransaction(txn, lsig)

# Step 6.3 Send
txid = algod_client.send_transaction(stxn)

# Step 6.4 Wait for ...
txinfo = wait_for_confirmation(algod_client, txid)


Current round is  19828561.
Waiting for round 19828561 to finish.
Waiting for round 19828562 to finish.
Transaction KMZZXPJKJZ2SJIN7NIT4COT4FJXRUQ6KABAWR2KL5H5M2GBCEM5A confirmed in round 19828563.


#### Now try the following
* Try different arguments that add up to 10
* Try incorrect arguments that do not add up to 10

#### Exercise 1
* Your MyAlgo account should do this:
* Create a quiz so that three numbers have to add up to a secret number (choose between 0 and 10)
* Fund the quiz
* Invite your neighbour to play the quiz

#### Exercise 2
* Create a quiz with *pythagorean triples*, like, for example, $3^2 + 4^2 = 5^5$
* The quiz pays out 10 micro Algos for every correct solution $a,b,c$ such that $a^2+b^2=c^2$

Solution `Btoi(Arg(0)) * Btoi(Arg(0))  + Btoi(Arg(1))*Btoi(Arg(1)) ==  Btoi(Arg(2))*Btoi(Arg(2))`

## Appendix: the story why there is only integer arithmetic on the blockchain

In [15]:
3.3 + 8.8 == 12.1

False

In [16]:
3.3 + 8.8

12.100000000000001